# Practice Session 06: PageRank

# 1. Read host names

In [ ]:
import io
import gzip
import csv
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
INPUT_NODES_FILENAME = "webspam_uk2007-nodes.csv.gz"
INPUT_EDGES_FILENAME = "webspam_uk2007-edges.csv.gz"

In [ ]:
name2id = {} #Creamos los 3 diccionarios
id2name = {} 
id2label = {}
with gzip.open(INPUT_NODES_FILENAME, "rt", encoding="utf-8") as input_file:
    reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
    for record in reader:
        name2id[record["hostname"]] = int(record["nodeid"])#Key hostname. devuelve la id del nodo que se corresponde a ese nombre
        id2name[int(record["nodeid"])] = record["hostname"]#Key id. devuelve el nombre que corresponde a esa id 
        id2label[int(record["nodeid"])] = record["label"]#Key id. devuelve el label que corresponde a esa id

In [ ]:
# Leave as-is

print("%s: %s" % (id2name[873], id2label[873]))
print("%s: %s" % (id2name[105715], id2label[105715]))
print("Number of hosts: %s" % len(id2name))

bbc.co.uk: nonspam
www.top-mobile-phones.co.uk: spam
Number of hosts: 114529


In [ ]:
spam = 0
nonspam = 0
unlabeled = 0
#miramos las labels de todos los nodos y contamos las veces que aparece spam, las veces que aparece nonspam y las veces que aparace unlabeled.
for i in range(len(id2label)):
    if id2label[i] == 'spam':
        spam+=1
    elif id2label[i] == 'nonspam':
        nonspam+=1
    elif id2label[i] == 'unlabeled':
        unlabeled+=1
print("%d, %d, %d" % (spam, nonspam, unlabeled))

344, 5709, 108476


In [ ]:
with gzip.open(INPUT_EDGES_FILENAME, "rt", encoding="utf-8") as input_file:
    reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
    spammywords = ['escort', 'xx', 'girl', 'credit', 'mortgage', 'finance', 'debt', 'loan']#Lista de palabras que si aparecen se consideraran spam
    g = nx.DiGraph()#creamos un grafo dirigido
    for record in reader:
        for word in spammywords:#revisamos si alguna de las palabras de la lista esta en el nodo source o en el de destino, ademas miramos que la label del source y del destino no sea unlabeled.
            if((word in id2name[int(record["source"])] or word in id2name[int(record["destination"])])and id2label[int(record["source"])] != "unlabeled" and id2label[int(record["destination"])]!= "unlabeled"):
                g.add_edge(id2name[int(record["source"])], id2name[int(record["destination"])])#Añadimos el enlace al grafo
print(g.number_of_nodes())

57


In [ ]:
# Leave this code as-is, or modify slightly

colors = []
hostname_converted = {}

for hostname in g.nodes():
    # Assign colors to nodes according to spam/nonspam labels
    if id2label[name2id[hostname]] == 'spam':
        colors.append('red')
    elif id2label[name2id[hostname]] == 'nonspam':
        colors.append('lightgreen')
    else:
        colors.append('white')
    
    # Shorten the hostnames to generate labels    
    label = hostname.replace("www.", "").replace(".uk", "")
    hostname_converted[hostname] = label
    
# Notice that if you re-run this cell the layout will be different every time
plt.figure(figsize=(20, 20))
plt.axis('off')
pos = nx.spring_layout(g)
nx.draw_networkx(g, pos, with_labels=True, node_size=400, node_color=colors, labels=hostname_converted)

La gran mayoria de enlaces de spam de caracter economico provienen de la pagina stopbyfinance.co, por lo que está pagina es el nodo de mayor grado y de igual manera la que mas "spam" proporciona a la red de la lista.
Luego vemos que hay mas de una componente conexa, hay 2 grupos de paginas de caracter pornografico, por lo que contienen en sus nombres. En una de estas componentes hay un nodo que predomina también y la mayoria de enlaces salen o van del nodo con nombre de pagina escortnet.co.
Hay muchas paginas o nodos con grado 1 por lo cual solo estan relacionadas con una sola pagina.

# 2. Compute the degree of each node

In [ ]:
# Leave this code as-is

id2degree = {}
N = len(id2name)
for nodeid in range(N):
    id2degree[nodeid] = 0

In [ ]:
with gzip.open(INPUT_EDGES_FILENAME, "rt", encoding="utf-8") as input_file:
    reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
    for record in reader:
      id2degree[int(record['source'])] += 1 #Suma 1 cada vez que se encuentra esta posicion, si el source es 0 pues suma 1 y a la siguiente igual


In [ ]:
# Leave this cell as-is

for nodeid in [890, 1469, 105715]:
    print("%s: degree %d" % (id2name[nodeid], id2degree[nodeid]))

bc1.org.uk: degree 16
candycaine.skinthesun.co.uk: degree 22
www.top-mobile-phones.co.uk: degree 0


# 3. Compute PageRank

In [ ]:
# Leave this cell as-is

ITERATIONS = 20
ALPHA = 0.85

pagerank_aux = [0.0] * N
pagerank = [1.0/N] * N

In [ ]:

for iteration in range(ITERATIONS):
    i = 0
    with gzip.open(INPUT_EDGES_FILENAME, "rt", encoding="utf-8") as input_file:
      reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
      for record in reader:
        pagerank_aux[int(record['destination'])] += pagerank[int(record['source'])]/id2degree[int(record['source'])]#Aplicamos la formula del pagerank
      while i < N:
        pagerank[i] = ALPHA*pagerank_aux[i]+(1-ALPHA)*(1.0/N)
        i+=1
    pagerank_aux = [0.0] * N #Volvemos a poner el diccionario auxiliar a 0 para calcular correctamente el siguiente pagerank
  

# 4. Nodes with largest values of PageRank

In [ ]:
i = 0
rounded = [0.0] * N 
while i < N:
  rounded[i] = round(pagerank[i], 6)#Redondeamos cada valor del pagerank a 6 decimales
  i = i+1
hosts_by_score = sorted(enumerate(rounded), key=lambda x: x[1], reverse=True) #Ordenamos el pagerank de manera descendente
i = 0 
while i < 20:
  print("host id: %d, host name: %s, label: %s, score: %g" % (hosts_by_score[i][0], id2name[hosts_by_score[i][0]], id2label[hosts_by_score[i][0]], hosts_by_score[i][1]))#Imprimimos los 20 nodos con mas pagerank.
  i = i+1



1.   Porque es una pagina gubernamental del reino unido donde se informa de la legislación del país, por lo que muchas paginas te llevaran a esta para darte cualquier tipo de información, mismamente en todas las paginas puede haber algo que esté relacionado con la ley de privacidad la cual si te quieres informar te redigirá a esta web.
2.   La gran mayoria de webs son paginas de caracter gubernamental, luego se encontrarian las educacionales y finalmente las comerciales.



# 5. Run non-spam PageRank

In [ ]:
id2nsdegree = {}
N = len(id2name)
for nodeid in range(N):
    id2nsdegree[nodeid] = 0
with gzip.open(INPUT_EDGES_FILENAME, "rt", encoding="utf-8") as input_file:
    reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
    for record in reader:
        if id2label[int(record['source'])] != "spam" and id2label[int(record['destination'])] != "spam":
            id2nsdegree[int(record['source'])] += 1#Miramos el grado de los nodos que no sean spam contando las veces que aparecen estos nodos.


In [ ]:
# Leave this cell as-is

for nodeid in [890, 1469, 105715]:
    print("%s: normal degree %d nospam degree %d" % (id2name[nodeid], id2degree[nodeid], id2nsdegree[nodeid]))

In [ ]:
pagerankns_aux = [0.0] * N
pagerankns = [1.0/N] * N

In [ ]:
#Mismo proceso que en el pagerank pero esta vez lo realizamos solo con las palabras que no sean spam
for iteration in range(ITERATIONS):
    i = 0
    with gzip.open(INPUT_EDGES_FILENAME, "rt", encoding="utf-8") as input_file:
      reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
      for record in reader:
        if id2label[int(record['source'])] != "spam" and id2label[int(record['destination'])] != "spam":
          pagerankns_aux[int(record['destination'])] += pagerankns[int(record['source'])]/id2nsdegree[int(record['source'])]
      while i < N:
        pagerankns[i] = ALPHA*pagerankns_aux[i]+(1-ALPHA)*(1.0/N)
        i+=1
    pagerankns_aux = [0.0] * N

In [ ]:
#Mismo proceso que en el pagerank pero esta vez sin spam
i = 0
rounded = [0.0] * N 
while i < N:
  rounded[i] = round(pagerankns[i], 6)
  i = i+1
hosts_by_scorens = sorted(enumerate(rounded), key=lambda x: x[1], reverse=True)
i = 0 
while i < 20:
  print("host id: %d, host name: %s, label: %s, score: %g" % (hosts_by_scorens[i][0], id2name[hosts_by_scorens[i][0]], id2label[hosts_by_scorens[i][0]], hosts_by_scorens[i][1]))
  i = i+1

# 6. Compute spam gain

In [ ]:
spam_gain = [0.0]*N
spam_gain_rounded = [0.0]*N
for i in range(N):
  spam_gain[i] = pagerank[i]/pagerankns[i]#Miramos la ganancia por el hecho de ser spam de cada nodo
  spam_gain_rounded[i] = round(spam_gain[i], 2) #redondeamos a 2 decimales
spam_gain_ordered = sorted(enumerate(spam_gain_rounded), key=lambda x: x[1], reverse=True) #Ordenamos la ganancia de manera descendente

for j in range(50):
  #Imprimimos los 50 con más ganancia
    print("Host name: %s, spam gain: %g, host label: %s, " % (id2name[spam_gain_ordered[j][0]], spam_gain_ordered[j][1], id2label[spam_gain_ordered[j][0]])+ "pagerank: ""{:.2e}, ".format(pagerank[spam_gain_ordered[j][0]])+ "pagerank no-spam: ""{:.2e}".format(pagerankns[spam_gain_ordered[j][0]]) )



Porque hay muchas webs que no tienen relación con el spam, es decir no redirigen a paginas que són consideradas spam según nuestro algoritmo por lo que el pagerank teniendo en cuenta el spam y sin tenerlo en cuenta es el mismo valor por lo que la ganancia seria 0. Por otra parte vemos que la mayoria de webs que ganan mas contando el spam se debe a que són webs que propiamente són spam y redirigen a otras paginas de spam o otras paginas redirigen a estas.

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>